In [10]:
from tsplib95 import load
from aco_utils.ant_colony import AntColony
from numpy.random import random, uniform, randint

In [11]:
"""
    Runs a static-network version of MMAS
    @:param: file_name - location of graph file. Must be in .tsp format
"""
def static_tsp_mmas(file_name):
    network = load('data/' + file_name + '.tsp').get_graph()
    avg = 0.0
    n = network.number_of_nodes()
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            avg += network[i][j]['weight']
    avg = avg / network.number_of_edges()
    ac = AntColony(network=network, colony_size=15, ph_min=0, ph_max=avg, pheromone_evaporation_coeff=0.3, alpha=1, beta=2)
    best_ants = []

    ac.initialize()
    for i in range(100):
        ac.execute_once()
        best_ant = ac.best_ant()
        best_ants.append(best_ant)
        
    best_ant = min(best_ants, key=ac.fitness)
    best_ant_path = best_ant.path
    best_ant_path.append(best_ant_path[0])
    best_ant_fitness = ac.fitness(best_ant)
    
    print(best_ant_fitness)
    print(best_ant_path)
    
    with open('output/' + file_name + '.txt', 'w') as file:
        file.write(str(best_ant_fitness))
        file.write('\n')
        file.write(str(best_ant_path))

In [12]:
static_tsp_mmas('att48')
#10628

26328
[18, 44, 28, 19, 37, 7, 39, 5, 2, 45, 35, 24, 42, 41, 8, 15, 40, 33, 9, 38, 11, 47, 46, 31, 6, 17, 43, 48, 25, 14, 29, 10, 4, 20, 12, 22, 3, 21, 13, 1, 30, 27, 36, 34, 16, 23, 26, 32, 18]


In [13]:
static_tsp_mmas('eil51')
#426

1092
[21, 16, 50, 38, 32, 47, 4, 48, 36, 49, 44, 51, 46, 37, 5, 12, 40, 19, 41, 13, 24, 26, 43, 7, 39, 33, 45, 42, 22, 27, 34, 29, 9, 30, 3, 28, 8, 6, 2, 11, 10, 17, 35, 1, 15, 14, 25, 18, 31, 23, 20, 21]


In [25]:
static_tsp_mmas('burma14')
#3323

3530
[8, 1, 10, 2, 9, 11, 13, 7, 3, 14, 6, 12, 5, 4, 8]


In [15]:
"""
    Modifies the weight of a random edge from a given network
"""
def modify_network(network):
    n1 = randint(1, network.number_of_nodes() + 1)
    n2 = randint(1, network.number_of_nodes() + 1)
    while n1 != n2:
        n2 = randint(1, network.number_of_nodes() + 1)
    modification_coeff = uniform(0.5, 1.5)
    network[n1][n2]['weight'] = network[n1][n2]['weight'] * modification_coeff

In [16]:
"""
    Runs a dynamic-network version of MMAS
    @:param: file_name - location of graph file. Must be in .tsp format
    @:param: iteration_modification_coeff - determines how many iterations are modified
"""
def dynamic_tsp_mmas(file_name, iteration_modification_coeff):
    network = load('data/' + file_name + '.tsp').get_graph()
    avg = 0.0
    n = network.number_of_nodes()
    for i in range(1, n + 1):
        for j in range(1, n + 1):
            avg += network[i][j]['weight']
    avg = avg / network.number_of_edges()
    ac = AntColony(network=network, colony_size=20, ph_min=0, ph_max=avg, pheromone_evaporation_coeff=0.3, alpha=1, beta=2)
    best_ants = []

    ac.initialize()
    for i in range(100):
        ac.execute_once()
        best_ant = ac.best_ant()
        best_ants.append(best_ant)
        if random() < iteration_modification_coeff:
            best_ant_path = best_ant.path
            best_ant_path.append(best_ant.path[0])
            print(ac.fitness(min(best_ants, key=ac.fitness)))
            print(best_ant_path)
            with open('output/' + file_name + '-dynamic.txt', 'a+') as file:
                file.write(str(ac.fitness(min(best_ants, key=ac.fitness))))
                file.write('\n')
                file.write(str(best_ant_path))
                file.write('\n')
            for _ in range(5):
                n1 = randint(1, network.number_of_nodes() + 1)
                n2 = randint(1, network.number_of_nodes() + 1)
                while n1 == n2:
                    n2 = randint(1, network.number_of_nodes() + 1)
                modification_coeff = uniform(0.5, 1.5)
                old_weight = network[n1][n2]['weight']
                network[n1][n2]['weight'] = network[n1][n2]['weight'] * modification_coeff
                network[n2][n1]['weight'] = network[n1][n2]['weight'] * modification_coeff
                print('Network modified: edge (' + str(n1) + ', ' + str(n2) + '): old weight=' + str(old_weight) + ' new weight=' + str(network[n1][n2]['weight']))
                with open('output/' + file_name + '-dynamic.txt', 'a+') as file:
                    file.write('Network modified: edge (' + str(n1) + ', ' + str(n2) + '): old weight=' + str(old_weight) + ' new weight' + str(network[n1][n2]['weight']))
                    file.write('\n')
            print('\n')
            best_ants = []

In [17]:
dynamic_tsp_mmas('att48', 0.3)

18205
[9, 3, 14, 25, 21, 40, 16, 20, 37, 19, 17, 43, 6, 30, 27, 46, 36, 18, 7, 28, 44, 31, 38, 33, 15, 1, 11, 13, 23, 48, 5, 29, 26, 4, 10, 35, 45, 24, 32, 39, 47, 8, 41, 34, 22, 12, 2, 42, 9]
Network modified: edge (11, 32): old weight=737 new weight=1187.4898980533824
Network modified: edge (46, 5): old weight=1253 new weight=918.3871891470043
Network modified: edge (43, 19): old weight=154 new weight=188.4790796164441
Network modified: edge (34, 22): old weight=473 new weight=952.3903045242234
Network modified: edge (2, 11): old weight=1312 new weight=855.753484862296


18601
[14, 25, 23, 11, 13, 34, 8, 22, 16, 9, 38, 1, 40, 7, 33, 36, 28, 44, 18, 6, 27, 37, 43, 30, 19, 17, 15, 31, 46, 47, 5, 39, 32, 29, 26, 4, 35, 45, 21, 20, 12, 3, 41, 42, 24, 10, 48, 2, 14]
Network modified: edge (47, 43): old weight=842 new weight=1146.4216217950866
Network modified: edge (4, 43): old weight=2526 new weight=3562.707140696745
Network modified: edge (44, 39): old weight=1280 new weight=1795.293839

25184.56493870046
[47, 20, 33, 46, 30, 44, 7, 28, 25, 39, 2, 10, 35, 45, 34, 23, 9, 40, 3, 16, 41, 43, 27, 19, 6, 36, 12, 31, 48, 42, 24, 5, 14, 11, 22, 8, 1, 15, 4, 26, 17, 37, 18, 38, 21, 13, 29, 32, 47]
Network modified: edge (13, 25): old weight=169 new weight=64.87631800466598
Network modified: edge (42, 20): old weight=1409 new weight=509.7040401110471
Network modified: edge (29, 6): old weight=1826 new weight=2359.0310873045228
Network modified: edge (31, 11): old weight=660 new weight=584.3345743566416
Network modified: edge (39, 13): old weight=390 new weight=204.78983410501363


26056.587595323024
[44, 31, 20, 33, 7, 18, 30, 27, 43, 17, 46, 9, 16, 3, 34, 41, 32, 42, 35, 45, 24, 48, 47, 25, 23, 12, 40, 10, 2, 21, 39, 13, 36, 6, 37, 19, 29, 5, 11, 1, 8, 38, 4, 26, 28, 15, 22, 14, 44]
Network modified: edge (34, 11): old weight=522 new weight=872.746693167948
Network modified: edge (9, 4): old weight=2081 new weight=3581.160733729556
Network modified: edge (6, 38): old weight=70

In [18]:
dynamic_tsp_mmas('eil51', 0.3)

810
[3, 20, 35, 36, 50, 9, 49, 48, 7, 26, 8, 23, 43, 24, 21, 29, 38, 39, 4, 47, 32, 17, 18, 6, 14, 27, 1, 46, 51, 11, 41, 19, 42, 44, 15, 37, 12, 16, 2, 22, 28, 31, 40, 13, 25, 45, 33, 10, 30, 34, 5, 3]
Network modified: edge (3, 29): old weight=17 new weight=9.192382027327232
Network modified: edge (25, 30): old weight=52 new weight=34.17557495946021
Network modified: edge (39, 29): old weight=33 new weight=53.8105746000497
Network modified: edge (21, 38): old weight=18 new weight=39.05743534358975
Network modified: edge (20, 23): old weight=41 new weight=27.22774735321416


969
[43, 25, 5, 46, 51, 6, 34, 50, 9, 49, 10, 17, 37, 47, 11, 44, 19, 40, 4, 18, 23, 48, 2, 38, 29, 36, 35, 20, 39, 28, 8, 27, 22, 13, 41, 42, 45, 15, 16, 3, 12, 14, 7, 1, 24, 26, 31, 32, 21, 30, 33, 43]
Network modified: edge (2, 18): old weight=36 new weight=41.476237597825204
Network modified: edge (31, 17): old weight=47 new weight=26.21483205656073
Network modified: edge (2, 21): old weight=15 new weight=18.7

1026.615341297765
[23, 48, 27, 45, 33, 10, 9, 21, 51, 46, 17, 44, 42, 18, 13, 15, 47, 26, 31, 2, 29, 49, 40, 37, 14, 50, 34, 30, 35, 43, 22, 36, 20, 11, 32, 1, 28, 3, 25, 19, 41, 4, 38, 39, 16, 24, 7, 6, 5, 12, 8, 23]
Network modified: edge (31, 43): old weight=32 new weight=42.015856852602695
Network modified: edge (7, 47): old weight=32 new weight=45.92621442916609
Network modified: edge (6, 39): old weight=50 new weight=112.34128375276607
Network modified: edge (2, 36): old weight=24 new weight=24.21107295294978
Network modified: edge (20, 31): old weight=23 new weight=29.22374068990401


938.5533045080377
[16, 38, 27, 47, 46, 37, 15, 34, 21, 50, 49, 30, 51, 48, 22, 7, 23, 25, 41, 44, 19, 13, 4, 17, 33, 39, 9, 5, 29, 2, 32, 1, 28, 3, 8, 14, 18, 24, 43, 40, 42, 45, 26, 31, 35, 36, 11, 12, 6, 20, 10, 16]
Network modified: edge (10, 39): old weight=10 new weight=2.85985965396371
Network modified: edge (25, 46): old weight=25 new weight=20.68955770271343
Network modified: edge (7, 50): 

In [19]:
dynamic_tsp_mmas('burma14', 0.3)

3687
[12, 6, 7, 13, 14, 3, 2, 8, 1, 11, 9, 10, 5, 4, 12]
Network modified: edge (12, 8): old weight=508 new weight=433.63125397072497
Network modified: edge (1, 7): old weight=455 new weight=874.7734633034411
Network modified: edge (2, 9): old weight=311 new weight=360.09532251452066
Network modified: edge (13, 4): old weight=499 new weight=622.2620582586027
Network modified: edge (5, 11): old weight=947 new weight=714.6557780362929


3576
[5, 6, 12, 4, 3, 8, 1, 10, 9, 11, 7, 13, 2, 14, 5]
Network modified: edge (2, 7): old weight=507 new weight=699.6048338655542
Network modified: edge (6, 7): old weight=168 new weight=154.15786824777538
Network modified: edge (13, 2): old weight=417 new weight=186.49995322472
Network modified: edge (3, 4): old weight=289 new weight=283.9691780786802
Network modified: edge (6, 14): old weight=239 new weight=185.12969361970974


3989.6578214724955
[9, 11, 1, 2, 13, 7, 6, 12, 4, 5, 3, 8, 10, 14, 9]
Network modified: edge (7, 14): old weight=232 new weigh

2467.0103798666155
[4, 6, 12, 13, 7, 3, 14, 10, 2, 11, 9, 1, 8, 5, 4]
Network modified: edge (14, 11): old weight=464 new weight=125.53684189212427
Network modified: edge (5, 14): old weight=485.1785382834461 new weight=1057.4370031618687
Network modified: edge (2, 8): old weight=197 new weight=428.29631824780006
Network modified: edge (11, 4): old weight=768 new weight=525.0748594673213
Network modified: edge (2, 8): old weight=428.29631824780006 new weight=921.9425302349017


2423.095544568706
[14, 13, 7, 3, 11, 9, 1, 8, 5, 4, 6, 12, 2, 10, 14]
Network modified: edge (4, 3): old weight=612.467812094866 new weight=884.7634440056983
Network modified: edge (1, 2): old weight=153 new weight=216.78998570263954
Network modified: edge (6, 8): old weight=314.7959878574988 new weight=356.51288568136283
Network modified: edge (9, 11): old weight=85.98670220358925 new weight=33.23869728740817
Network modified: edge (5, 1): old weight=966 new weight=2145.366346726673


2555.2330966720606
[5, 13,